In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
c

Client Scheduler: tcp://127.0.0.1:46783 Dashboard: http://127.0.0.1:36434/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [3]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import numpy.ma as ma

today=date.today()

#https://pmall.univ-grenoble-alpes.fr/docs/auto_examples_code/windrose/windrose-exemple.html

In [4]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
    

In [5]:
data_dir = '/store/molines/NATL60/NATL60-CJM165-S/1h/'
gridfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_mesh_hgr.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [6]:
e1=dsgrid['e1t'][0]
e2=dsgrid['e2t'][0]


In [7]:
imin = {'AC' : 3088, 'NE' :3838, 'GS' : 994}
imax = {'AC' : 3386, 'NE' :4119, 'GS' : 1295}
jmin = {'AC' : 301, 'NE' :1783, 'GS' : 751}
jmax = {'AC' : 657, 'NE' :2239, 'GS' : 1155}


In [ ]:
for month in ['march', 'september']:
#for month in ['september']:
    
    if month == 'march':
        Date=['20130301','20130302','20130303','20130304','20130305','20130306','20130307','20130308','20130309','20130310','20130311','20130312','20130313','20130314','20130315','20130316','20130317','20130318','20130319','20130320','20130321','20130322','20130323','20130324','20130325','20130326','20130327','20130328','20130329','20130330']
    if month == 'september':
        Date=['20120901','20120902','20120903','20120904','20120905','20120906','20120907','20120908','20120909','20120910','20120911','20120912','20120913','20120914','20120915','20120916','20120917','20120918','20120919','20120920','20120921','20120922','20120923','20120924','20120925','20120926','20120927','20120928','20120929','20120930']

    for date in Date:
        print(date)
        
        year=date[0:4]
        month=date[4:6]
        day=date[6:]

        filename = sorted(glob.glob(data_dir+str(year)+'/NATL60-CJM165_y'+str(year)+'m'+str(month)+'d'+str(day)+'.1h_gridT.nc'))
        file=filename[0]
        print(file)

        ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1})
        dataT=ds['sosstsst']
        dataS=ds['sosaline']
        dataT_dx=dx_var(dataT,e1)
        dataS_dx=dx_var(dataS,e1)
        dataT_dy=dy_var(dataT,e1)
        dataS_dy=dy_var(dataS,e1)
        R=(dataT_dx + 1j * dataT_dy)/(dataS_dx + 1j * dataS_dy)
        magR=np.abs(R)
        magR_ma=ma.masked_invalid(magR)
        Tu=np.arctan(magR)
        
#        for box in ['AC','NE','GS']:
        for box in ['NE','GS']:

            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1h/turner-angle/NATL60-CJM165_magR_'+str(date)+'.1h_'+str(box)+'.nc'
            if not os.path.exists(filename): 
                
                print('compute magR for box '+str(box))
                magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
                Tu_box=Tu[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
                magR_box_nonan=magR_box[~np.isnan(magR_box)]
                Tu_box_nonan=Tu_box[~np.isnan(magR_box)]
                magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
                Tu_box_nonan_noinf=Tu_box_nonan[~np.isinf(magR_box_nonan)]
                magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
                Tu_box_nonan_noinf=Tu_box_nonan_noinf.reset_index('z')

                print('write /'+filename)
                dataset_mag=magR_box_nonan_noinf.to_dataset(name='magR')
                dataset_mag.attrs['global_attribute']= 'Stack of absolute value of ratio between temperature and salinity gradient of data for day '+str(date)
                dataset_mag.to_netcdf(path=filename,mode='w')
    
            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1h/turner-angle/NATL60-CJM165_Tu_'+str(date)+'.1h_'+str(box)+'.nc'
            if not os.path.exists(filename):
                
                print('compute magR for box '+str(box))
                magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
                Tu_box=Tu[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
                magR_box_nonan=magR_box[~np.isnan(magR_box)]
                Tu_box_nonan=Tu_box[~np.isnan(magR_box)]
                magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
                Tu_box_nonan_noinf=Tu_box_nonan[~np.isinf(magR_box_nonan)]
                magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
                Tu_box_nonan_noinf=Tu_box_nonan_noinf.reset_index('z')

                print('write '+filename)
                dataset_mag=Tu_box_nonan_noinf.to_dataset(name='Tu')
                dataset_mag.attrs['global_attribute']= 'Stack of turner angle of data for day '+str(date)
                dataset_mag.to_netcdf(path=filename,mode='w')


20130301
/store/molines/NATL60/NATL60-CJM165-S/1h/2013/NATL60-CJM165_y2013m03d01.1h_gridT.nc
compute magR for box NE


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:54042 remote=tcp://127.0.0.1:46783>
distributed.nanny - WARNING - Worker process 23559 was killed by signal 9
distributed.nanny - WARNING - Worker process 23561 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 23565 was killed by signal 9
distributed.nanny - WARNING - Worker process 23563 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 23571 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 23567 was killed by signal 9
distributed.nanny - WARNING - Worker process 23569 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WA

write //scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1h/turner-angle/NATL60-CJM165_magR_20130301.1h_NE.nc
compute magR for box NE


distributed.nanny - WARNING - Worker process 53819 was killed by signal 9
distributed.nanny - WARNING - Worker process 53815 was killed by signal 9
distributed.nanny - WARNING - Worker process 53887 was killed by signal 9
distributed.nanny - WARNING - Worker process 53895 was killed by signal 9
distributed.nanny - WARNING - Worker process 53922 was killed by signal 9
distributed.nanny - WARNING - Worker process 54019 was killed by signal 9
distributed.nanny - WARNING - Worker process 54117 was killed by signal 9
distributed.nanny - WARNING - Worker process 54027 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARN

write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1h/turner-angle/NATL60-CJM165_Tu_20130301.1h_NE.nc
compute magR for box GS
